# Basic Response Prediction Using an MLP Architecture

In [ ]:
import itertools

import numpy as np
import pandas as pd

from pathlib import Path
from tensorflow import keras

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from cdrpy.datasets import Dataset
from cdrpy.data.preprocess import normalize_responses
from cdrpy.mapper import BatchedResponseGenerator
from cdrpy.metrics import tf_metrics

In [ ]:
SEED = 1771

## 1. Data preparation

### Loading the drug response dataset

We can load a saved `cdrpy` datasest using the `Dataset.load` method. This demo dataset consists of harmonized cell line pharmacogenomic data retreived from the Genomics of Drug Sensitivity in Cancer and the Cell Model Passports data resources.

In this dataset, cell lines are represented by log-transformed TPM gene expression values for 1771 cancer-relevant genes. Drugs are represented as 512 bit Morgan Fingerprints. Drug responses correspond to the natural log of the half-maximal inhibitory concentration (ln(IC50)) values.

In [ ]:
D = Dataset.load("../../data/datasets/temp/demo.h5")
print(D)

In [ ]:
# NOTE: add some data exploration -> how many features do we have
# for expression and morgan fingerprints

### Splitting the data

In precision oncology, we need to predict response in never-before-seen patients. To mimic this setting, we will use scikit-learn's `train_test_split` function to generate a train/test split in which all drug responses for a given cell line are held out for evaluation.

In [ ]:
cell_ids = D.cell_meta.index
cell_groups = D.cell_meta["cancer_type"]  # stratify by cancer type
test_size = 0.1

train_cell_ids, test_cell_ids = train_test_split(
    cell_ids,
    stratify=cell_groups,
    test_size=test_size,
    random_state=SEED,
)

train_cell_ids, val_cell_ids = train_test_split(
    train_cell_ids,
    stratify=cell_groups.loc[train_cell_ids],
    test_size=test_size,
    random_state=SEED,
)

train_ds = D.select_cells(train_cell_ids, name="train")
val_ds = D.select_cells(val_cell_ids, name="val")
test_ds = D.select_cells(test_cell_ids, name="test")

### Data preprocessing

To avoid data leakage, several preprocessing steps should be completed *after* splitting the data in train/validation/test sets.

In this case, we will use scikit-learn's `StandardScaler` class to normalize gene expression values.

In [ ]:
ge_data = train_ds.cell_encoders["exp"].data

ge_scaler = StandardScaler().fit(ge_data.loc[train_cell_ids])
ge_data[:] = ge_scaler.transform(ge_data)

We also need to normalize the raw drug response observations. `cdrpy` provides the `normalize_responses` helper function for this purpose. This function accepts an optional parameter, `norm_method`. When `norm_method` is set to `grouped`, normalization is performed per drug, reducing bias in overall performance estimates by variability in removing drug-specific effective concentration range from the data.

In [ ]:
train_ds, val_ds, test_ds = normalize_responses(
    train_ds, val_ds, test_ds, norm_method="grouped"
)

## 2. Creating the model

As an example, we will construct a simple model using three multi-layer perceptrons (MLPs).

In [ ]:
ge_input_dim = train_ds.cell_encoders["exp"].shape[-1]
fp_input_dim = train_ds.drug_encoders["mol"].shape[-1]

x_ge = ge_input = keras.Input((ge_input_dim,))
x_ge = keras.layers.Dense(128, activation="relu")(x_ge)
x_ge = keras.layers.Dense(64, activation="relu")(x_ge)

x_fp = fp_input = keras.Input((fp_input_dim,))
x_fp = keras.layers.Dense(128, activation="relu")(x_fp)
x_fp = keras.layers.Dense(64, activation="relu")(x_fp)

x = keras.layers.Concatenate()([x_ge, x_fp])
x = keras.layers.Dense(64, activation="relu")(x)
x = keras.layers.Dense(32, activation="relu")(x)
x_out = keras.layers.Dense(1, activation="linear")(x)

In [ ]:
model = keras.Model([ge_input, fp_input], x_out)
model.summary()

## 3. Training and evaluation

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(1e-4),
    loss="mean_squared_error",
    metrics=[tf_metrics.pearson],
)

Train the model.

In [ ]:
generator = BatchedResponseGenerator(D, batch_size=128)

In [ ]:
train_gen = generator.flow_from_dataset(train_ds, shuffle=True, seed=SEED)
val_gen = generator.flow_from_dataset(val_ds)

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(
    monitor="val_loss", patience=5, restore_best_weights=True
)

In [ ]:
history = model.fit(
    train_gen, epochs=50, validation_data=val_gen, verbose=2, callbacks=[early_stopping]
)